In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

from IPython.display import clear_output
from datetime import datetime

In [2]:
param_date = '20240726'

In [3]:
import os

list_file = sorted(list(os.walk(f'./{param_date}/4.type_2'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

list_file

['group_1.Apparel & Gifts.xlsx',
 'group_10.Electrical-Switch & Relay.xlsx',
 'group_11.Engine.xlsx',
 'group_12.Exhaust & Emission.xlsx',
 'group_13.Fuel & Air.xlsx',
 'group_14.Garage Equipment.xlsx',
 'group_15.Hardware.xlsx',
 'group_16.Heat & Air Conditioning.xlsx',
 'group_17.Hoses%2FLines & Clamps.xlsx',
 'group_18.Ignition.xlsx',
 'group_19.Interior.xlsx',
 'group_2.Belt Drive.xlsx',
 'group_20.Literature.xlsx',
 'group_21.Steering.xlsx',
 'group_22.Suspension.xlsx',
 'group_23.Transmission-Automatic.xlsx',
 'group_24.Transmission-Manual.xlsx',
 'group_25.Wheel.xlsx',
 'group_26.Wiper & Washer.xlsx',
 'group_3.Body & Lamp Assembly.xlsx',
 'group_4.Brake & Wheel Hub.xlsx',
 'group_5.Cooling System.xlsx',
 'group_6.Drivetrain.xlsx',
 'group_7.Electrical.xlsx',
 'group_8.Electrical-Bulb & Socket.xlsx',
 'group_9.Electrical-Connector.xlsx']

In [4]:
for file in list_file[list_file.index('group_2.Belt Drive.xlsx'):]:

    # = = = = = = = = = = = = = = =

    import pandas as pd

    output_correct, output_error = pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./{param_date}/4.type_2/{file}',
                           header=0,
                           dtype=str).fillna('')

    input_['Group No'] = input_['Group No'].astype(int)
    input_['Type No'] = input_['Type No'].astype(int)
    input_['Manufacturer No'] = input_['Manufacturer No'].astype(int)

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =

    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_error

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'

            # = = = = = = = = = = = = = = =

            import requests

            import sys
            sys.path.append('../../00.Tools')
            from crawler_configuration import get_header, get_proxy

            from bs4 import BeautifulSoup
            from lxml import etree

            import re

            try:
                b = 0
                while True:

                    while True:
                        b += 1
                        try:
                            resp = requests.get(input_.loc[a, 'Tab 2 Url'],
                                                headers=get_header(),
                                                proxies=get_proxy(),
                                                timeout=(10, 10))
                            break
                        except:
                            continue

                    if resp.status_code == 500:
                        break
                    elif resp.status_code == 200:
                        soup = BeautifulSoup(resp.text, 'lxml')
                        html = etree.HTML(str(soup))
                        try:
                            title = re.sub(r' +', ' ', html.xpath('//div[@id="nav_o[1]"]/div[@class="inner"]/div/div[@style="display: inline-block; padding: 0em 5em 0em 5em; "]/text()')[0]).strip()

                            if html.xpath('//select[@id="manufacturer_partsearch_007"]/option[@selected]/@value')[0] == input_.loc[a, 'Manufacturer'] and html.xpath('//select[@id="partgroup_partsearch_007"]/option[@selected]/@value')[0] == input_.loc[a, 'Group'] and html.xpath('//select[@id="parttype_partsearch_007"]/option[@selected]/@value')[0] == input_.loc[a, 'Type Code'] and html.xpath('//select[@id="parttype_partsearch_007"]/option[@selected]/text()')[0] == input_.loc[a, 'Type'] or re.sub(r' +', ' ', f'''{input_.loc[a, 'Manufacturer']} {input_.loc[a, 'Type']} Part Numbers''').strip() == title:
                                break
                        except:
                            continue

                # = = = = = = = = = = = = = = =

                list_json = html.xpath('//div[@id="navchildren[1]"]/div[contains(@id, "nav[")]/input[contains(@id, "jsn[")]/@value')

                # = = = = = = = = = = = = = = =

                import json

                list_part_number = [json.loads(json_)['partnumber'] for json_ in list_json]

                # = = = = = = = = = = = = = = =

                list_url = [f'https://www.rockauto.com{href.strip()}' for href in html.xpath('//div[@id="navchildren[1]"]/div[contains(@id, "nav[")]/div[contains(@id, "nav_o[")]/div/table/tr/td[@class="nlabel nlbl-docolumns"]/a/@href')]

                # = = = = = = = = = = = = = = =

                if not list_part_number:
                    raise

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame({'Group No': input_.loc[a, 'Group No'],
                                        'Group': input_.loc[a, 'Group'],
                                        'Type No': input_.loc[a, 'Type No'],
                                        'Type': input_.loc[a, 'Type'],
                                        'Type Code': input_.loc[a, 'Type Code'],
                                        'Manufacturer No': input_.loc[a, 'Manufacturer No'],
                                        'Manufacturer': input_.loc[a, 'Manufacturer'],
                                        'Tab 2 Url': input_.loc[a, 'Tab 2 Url'],
                                        'SKU Count': len(list_part_number),
                                        'Part No': [i+1 for i in range(len(list_part_number))],
                                        'Part Number': list_part_number,
                                        'Url': list_url})

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                if crawler_status == 'error':
                    df_temp = pd.DataFrame([{'Group No': input_.loc[a, 'Group No'],
                                             'Group': input_.loc[a, 'Group'],
                                             'Type No': input_.loc[a, 'Type No'],
                                             'Type': input_.loc[a, 'Type'],
                                             'Type Code': input_.loc[a, 'Type Code'],
                                             'Manufacturer No': input_.loc[a, 'Manufacturer No'],
                                             'Manufacturer': input_.loc[a, 'Manufacturer'],
                                             'Tab 2 Url': input_.loc[a, 'Tab 2 Url']}])

                    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and work.qsize() == 0 or work.qsize() % 1_000 == 0 and work.qsize() != 0:
                clear_output()

            print(f'''[状态：{crawler_status}，尝试次数：{b}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} > {input_.loc[a, 'Type No']}.{input_.loc[a, 'Type']} > {input_.loc[a, 'Manufacturer No']}.{input_.loc[a, 'Manufacturer']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(20):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct = output_correct.sort_values(by=['Group No', 'Type No', 'Manufacturer No', 'Part No'],
                                                    ascending=[True, True, True, True],
                                                    ignore_index=True)
        output_correct.to_excel(f'''./{param_date}/5.part_number/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['Group No', 'Type No', 'Manufacturer No'],
                                                ascending=[True, True, True],
                                                ignore_index=True)
        output_error.to_excel(f'''./{param_date}/5.part_number/0.error/{file.removesuffix('.xlsx')}-part_number_error.xlsx''', index=False)
        print('爬虫存在error')
        print()
    print('Done ~')

[状态：ok，尝试次数：2] - 100.00% - group_9.Electrical-Connector > 128.Door Jamb Switch Connector > 6.STANDARD MOTOR PRODUCTS
[剩余数量：1000] - [当前时间：18:39:19]

[状态：ok，尝试次数：2] - 100.00% - group_9.Electrical-Connector > 129.Door Lock Actuator Connector > 7.WVE
[剩余数量：999] - [当前时间：18:39:19]

[状态：ok，尝试次数：2] - 100.00% - group_9.Electrical-Connector > 128.Door Jamb Switch Connector > 9.WVE
[剩余数量：998] - [当前时间：18:39:19]

[状态：ok，尝试次数：1] - 100.00% - group_9.Electrical-Connector > 119.Diesel Glow Plug Connector > 7.SKP
[剩余数量：997] - [当前时间：18:39:19]

[状态：ok，尝试次数：1] - 100.00% - group_9.Electrical-Connector > 130.Door Lock Switch Connector > 6.WVE
[剩余数量：996] - [当前时间：18:39:20]

[状态：ok，尝试次数：1] - 100.00% - group_9.Electrical-Connector > 130.Door Lock Switch Connector > 5.UNITED MOTOR PRODUCTS
[剩余数量：995] - [当前时间：18:39:22]

[状态：ok，尝试次数：1] - 100.00% - group_9.Electrical-Connector > 129.Door Lock Actuator Connector > 5.STANDARD MOTOR PRODUCTS
[剩余数量：994] - [当前时间：18:39:22]

[状态：ok，尝试次数：2] - 100.00% - group_9.Electrical-Co